# Using the Progress Callbacks with CPLEX Optimizer

This tutorial includes everything you need to set up decision optimization engines, build a mathematical programming model, then use the progress callbacks to follow the progress, capture the intermediate solutions and stop the solve on your own criteria.

Table of contents:

-  [Describe the business problem](#Describe-the-business-problem:--Games-Scheduling-in-the-National-Football-League)
*  [How decision optimization (prescriptive analytics) can help](#How--decision-optimization-can-help)
*  [Use decision optimization](#Use-decision-optimization)
    *  [Step 1: Set up the prescriptive model](#Step-1:-Set-up-the-prescriptive-model)
    *  [Step 2: Track the CPLEX progress](#Step-2:-Track-the-CPLEX-progress)
*  [Summary](#Summary)
****

## How  decision optimization can help

* Prescriptive analytics (decision optimization) technology recommends actions that are based on desired outcomes.  It takes into account specific scenarios, resources, and knowledge of past and current events. With this insight, your organization can make better decisions and have greater control of business outcomes.  

* Prescriptive analytics is the next step on the path to insight-based actions. It creates value through synergy with predictive analytics, which analyzes data to predict future outcomes.  

* Prescriptive analytics takes that insight to the next level by suggesting the optimal way to handle that future situation. Organizations that can act fast in dynamic conditions and make superior decisions in uncertain environments gain a strong competitive advantage.  
<br/>

<u>With prescriptive analytics, you can:</u> 

* Automate the complex decisions and trade-offs to better manage your limited resources.
* Take advantage of a future opportunity or mitigate a future risk.
* Proactively update recommendations based on changing events.
* Meet operational goals, increase customer loyalty, prevent threats and fraud, and optimize business processes.



## Use decision optimization

### Step 1: Set up the prescriptive model

First write a scalable model to show how to use the progress callback API.

This model is not important: it is a model that can take a very long time to solve and will find multiple intermediate solutions

In [1]:
from docplex.mp.model import Model

def build_hearts(r, **kwargs):
    # initialize the model
    mdl = Model('love_hearts_%d' % r, **kwargs)

    # the dictionary of decision variables, one variable
    # for each circle with i in (1 .. r) as the row and
    # j in (1 .. i) as the position within the row    
    idx = [(i, j) for i in range(1, r + 1) for j in range(1, i + 1)]
    a = mdl.binary_var_dict(idx, name=lambda idx_tuple: "a_%d_%d" % (idx_tuple[0], idx_tuple[1]))

    # the constraints - enumerate all equilateral triangles
    # and prevent any such triangles being formed by keeping
    # the number of included circles at its vertexes below 3

    # for each row except the last
    for i in range(1, r):
        # for each position in this row
        for j in range(1, i + 1):
            # for each triangle of side length (k) with its upper vertex at
            # (i, j) and its sides parallel to those of the overall shape
            for k in range(1, r - i + 1):
                # the sets of 3 points at the same distances clockwise along the
                # sides of these triangles form k equilateral triangles
                for m in range(k):
                    u, v, w = (i + m, j), (i + k, j + m), (i + k - m, j + k - m)
                    mdl.add_constraint(a[u] + a[v] + a[w] <= 2)

    mdl.maximize(mdl.sum(a))
    return mdl

It is a useful approach to demonstrate how to listen to CPLEX during its progress, as it contains multiple intermediate solutions.

### Step 2: Track the CPLEX progress

#### Basic track of the search progress
Use a <i>TextProgressListener</i> to follow the CPLEX search. It will print on the standard output each time an intermediate solution is found.

In [2]:
m5 = build_hearts(5)

from docplex.mp.progress import TextProgressListener
unfiltered_texter = TextProgressListener(filtering=False)
m5.add_progress_listener(unfiltered_texter)

#### Solve with Decision Optimization

Depending on the size of the problem, the solve stage might fail and require the Commercial Edition of CPLEX engines, which is included in the premium environments in Watson Studio.

You will get the best solution found after ***n*** seconds, because of a time limit parameter.

In [3]:
m5.solve(clean_before_solve=True)

  1+: Best Integer=8.0000, Best Bound=10.0000, gap=25.00%, nodes=0/1 [0.1s]
  2+: Best Integer=8.0000, Best Bound=10.0000, gap=25.00%, nodes=0/1 [0.2s]
  3+: Best Integer=8.0000, Best Bound=9.3266, gap=16.58%, nodes=0/1 [0.2s]
  4+: Best Integer=8.0000, Best Bound=9.0636, gap=13.29%, nodes=0/1 [0.2s]


docplex.mp.solution.SolveSolution(obj=8,values={a_1_1:1,a_2_1:1,a_3_1:1,..

Now remove the listener.

In [4]:
m5.remove_progress_listener(unfiltered_texter)

#### Track and store all intermediate solutions

You just need to subclass the <i>SolutionListener</i> object and specialize the <i>notify_*</i> methods if needed.
Here you will store all solutions all along the way.

In [5]:
from docplex.mp.progress import SolutionListener
class MyProgressListener(SolutionListener):
    def __init__(self, model):
        SolutionListener.__init__(self, model)
        self.solutions = []
 
    def notify_solution(self, s):
        SolutionListener.notify_solution(self, s)
        self.solutions.append(self.current_solution)
 
    def get_solutions(self):
        return self.solutions
    
keeper = MyProgressListener(m5)
m5.add_progress_listener(keeper)

In [6]:
m5.solve(clean_before_solve=True)

docplex.mp.solution.SolveSolution(obj=8,values={a_1_1:1,a_2_1:1,a_3_1:1,..

You have stored <i>docplex.mp.solution.SolveSolution</i> objects.
You can iterate on them to query the objective values, the values of each variables...

In [7]:
for s in keeper.get_solutions():
     print(s.objective_value)

8.0
8.0
8.0
8.0


#### Implement our own aborter
You might want to abort the CPLEX solution according to your own criteria.
For example, when the gap is converging very slowly, it might be a good idea to stop and use the last solution instead of waiting forever.

You just need to subclass the <i>ProgressListener</i> and specialize the <i>notify_*</i> methods.

In [8]:
from docplex.mp.progress import ProgressListener
class AutomaticAborter(ProgressListener):
    def __init__(self, max_no_solutions=10):
        self.last_incumbent_obj = -999999999
        self.nb_solutions = 0
        self.nb_non_improving = 0
        self.max_non_improving = max_no_solutions

    def notify_progress(self, progress_data):
        super(AutomaticAborter, self).notify_progress(progress_data)
        last_obj = self.last_incumbent_obj
        if progress_data.has_incumbent:
            if last_obj is None or progress_data.current_objective >= last_obj + 1e-5:
                self.nb_solutions += 1
                self.nb_non_improving = 0
                print('----> #solutions={}'.format(self.nb_solutions))
            else:

                # non improving move
                self.nb_non_improving += 1
                print('----> #non improving solutions={}'.format(self.nb_non_improving))
            self.last_incumbent_obj = progress_data.current_objective
        else:
            self.nb_non_improving += 1
        if self.nb_non_improving >= self.max_non_improving:
            if not self.has_aborted():
                print('!! aborting cplex, #solutions={0}, #non-improving: {1}'.format(self.nb_solutions,
                                                                                              self.nb_non_improving))
            self.abort()

Now build a bigger problem with more solutions.

In [9]:
love14 = build_hearts(11)

In [10]:
auto_abort = AutomaticAborter(max_no_solutions=50)
love14.add_progress_listener(auto_abort)

love14.solve()

----> #solutions=1
----> #non improving solutions=1
----> #non improving solutions=2
----> #non improving solutions=3
----> #non improving solutions=4
----> #non improving solutions=5
----> #non improving solutions=6
----> #non improving solutions=7
----> #non improving solutions=8
----> #non improving solutions=9
----> #non improving solutions=10
----> #non improving solutions=11
----> #non improving solutions=12
----> #non improving solutions=13
----> #non improving solutions=14
----> #non improving solutions=15
----> #non improving solutions=16
----> #non improving solutions=17
----> #non improving solutions=18
----> #non improving solutions=19
----> #non improving solutions=20
----> #non improving solutions=21
----> #non improving solutions=22
----> #non improving solutions=23
----> #non improving solutions=24
----> #non improving solutions=25
----> #non improving solutions=26
----> #non improving solutions=27
----> #non improving solutions=28
----> #non improving solutions=29
----

docplex.mp.solution.SolveSolution(obj=22,values={a_1_1:1,a_11_11:1,a_2_2..

## Summary


You have learned how to set up and use the IBM Decision Optimization CPLEX Modeling for Python to formulate a Mathematical Programming model and track its progress.

#### References
* <a href="https://rawgit.com/IBMDecisionOptimization/docplex-doc/master/docs/index.html" target="_blank" rel="noopener noreferrer">Decision Optimization CPLEX Modeling for Python documentation</a>
* <a href="https://dataplatform.cloud.ibm.com/docs/content/getting-started/welcome-main.html?audience=wdp&context=wdp" target="_blank" rel="noopener noreferrer">Watson Studio documentation</a>
* Need help with DOcplex or to report a bug? Go to <a href="https://developer.ibm.com/answers/smartspace/docloud" target="_blank" rel="noopener noreferrer">https://developer.ibm.com/answers/smartspace/docloud</a>


<hr>
Copyright © 2017-2018 This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>